# Retrieve student responses to New Quizzes

Author: Matthew Ford <mford@cornell.edu>,
Postdoctoral Associate in Mechanical & Aerospace Engineering,
Cornell Active Learning Initiative

The New Quizzes feature in Canvas is still under development (and has been foreeeeever), and lacks basic functionality to export a report of student responses. New Quizzes is implemented as an external tool, so results are not available through the documented Canvas API.

However, New Quizzes apparently uses a RESTful API very similar to the standard Canvas API, and can be reverse-engineered by monitoring requests made by your browser when viewing New Quiz results. __This script uses a combination of the standard Canvas API and the "secret" New Quiz API to retrieve student responses automatically__.

This script produces 4 output files:

- `course_<num>_students.csv`: Table of students with user_id, net_id, and name.
- `quiz_<num>_submissions.csv`: Table of quiz submissions, one row per submission. Students are identified by user_id.
- `quiz_<num>_responses.csv`: Table of student _responses_ to quiz items. Data is in "long format," i.e. one row per item. Data can easily be pivoted to obtain a table of student responses in which each column represents a different quiz item. Responses are identified by a unique response_id.
- `quiz_<num>_item_options.csv`: Table of answer options for each multiple-choice item, including item text response_id and response text.

These tables can be easily pivoted, joined, and manipulated to get data into whatever convenient format. For example:

`quiz_x_responses.csv >> LEFT JOIN with quiz_x_item_options.csv ON item_id >> PIVOT item_text to columns`

would give you table with one row per submission and one column per quiz item with the student response _text_ in each cell.

In [ ]:
from canvasapi import Canvas
import csv
import re
from urllib.parse import urlparse, parse_qs
import requests
import json

from IPython.display import JSON

## Load course data from Canvas API

- Create your own API key in Canvas/Account/Settings/Approved Integrations
- Put your API key in a text file called API_TOKEN.txt in the same directory as this script

In [ ]:
with open('API_TOKEN.txt', 'r') as f:
    api_token = f.readline()

canvas = Canvas('https://canvas.cornell.edu', api_token)

# List all your courses
for c in canvas.get_courses():
    print(c)

## Choose desired course by ID

- Enter the ID number of the course from the list above

In [ ]:
course_id = 

#### Download a list of students with net_ids

In [ ]:
course = canvas.get_course(course_id)
print(course)

with open('course_{:d}_students.csv'.format(course_id), 'w', newline='') as csv_file:

    fout = csv.DictWriter(csv_file,
                          fieldnames=['user_id', 'net_id', 'name'])
    fout.writeheader()

    n_students = 0
    for u in course.get_users(enrollment_type='student', include=['email']):
        fout.writerow({
            'user_id': u.id,
            'net_id': re.search('([a-z0-9]+)@cornell.edu', u.email).group(1),
            'name': u.name
        })
        n_students += 1
        print('{:d}'.format(n_students), end='\r', flush=True)

print('Found records for {:d} students'.format(n_students))

## Choose an assignment from the list below

In [ ]:
print('\nAssignments:')
for a in course.get_assignments():
    print(a)

## Enter desired course ID below

In [ ]:
assignment_id = 

In [ ]:
quiz = course.get_assignment(assignment_id)

# Uncomment the code below for Psych 1101 in which the quizzes were all named "Quiz <X> - ..."
# quiz_num = int(re.search('Quiz ([0-9]+)', quiz.name).group(1))
# quiz_name = 'quiz_{:02d}'.format(quiz_num)

# Uncomment the line below to use the assignment_id as the quiz name.
quiz_name = 'quiz_{:d}'.format(assignment_id)

print(quiz_name)

## Download quiz submission metadata

Creates a file `quiz_<assignment_id>_submissions.csv` with the following columns:

quiz_id | student_id | submission_id | attempt | submission_time | participant_session_id | quiz_session_id

In [ ]:
with open('{:s}_submissions.csv'.format(quiz_name), 'w', newline='') as csv_file,\
     open('{:s}_log.txt'.format(quiz_name), 'w') as log:

    log.write('Reading quiz submissions...\n')
    
    fout = csv.DictWriter(csv_file,
                          fieldnames=['user_id', 'assignment_id',
                                      'submission_id', 'attempt', 'submission_time',
                                      'participant_session_id', 'quiz_session_id'])
    fout.writeheader()
    
    n_subs = 0
    n_missing = 0
    for i, s in enumerate(quiz.get_submissions()):
        
        if not s.missing:
        
            try:
                quiz_session = parse_qs(urlparse(s.external_tool_url).query)

                fout.writerow({
                    'user_id': s.user_id,
                    'assignment_id': s.assignment_id, 
                    'submission_id': s.id,
                    'attempt': s.attempt,
                    'submission_time': s.submitted_at_date,
                    'participant_session_id': quiz_session['participant_session_id'][0],
                    'quiz_session_id': quiz_session['quiz_session_id'][0]})
                n_subs += 1
            except:
                msg = 'ERROR processing submission for User ID: {:d}, submission ID: {:d}'\
                    .format(s.user_id, s.id)
                print(msg)
                log.write(msg + '\n')
        else:
            msg = 'Missing submission for User ID: {:d}, submission ID: {:d}'.format(s.user_id, s.id)
            print(msg)
            log.write(msg + '\n')
            n_missing += 1

    print('Wrote submission metadata for {:d} submissions'.format(n_subs))
    print('Missing {:d} submissions'.format(n_missing))

    log.write('Wrote submission metadata for {:d} submissions\n'.format(n_subs))
    log.write('Missing {:d} submissions\n\n'.format(n_missing))

## Obtain a "Bearer" authorization token

- Run the code snipped below and click on the URL that appears (in Google Chrome)
- Open the Developer Tools, go to the Network tab, and refresh the page
- Click on "XHR" to see only AJAX requests
- Find the request called "grade" and locate the "authorization" field in Request Headers
- Copy the long string after "Bearer " and save to a file called "BEARER.txt" in the same directory as this file

The bearer token expires in 60 minutes, so make sure you complete the rest of the code within that time.

In [ ]:
# Get first valid submission
for s in quiz.get_submissions():
    if not s.missing:
        break

# Get submission information
quiz_session = parse_qs(urlparse(s.external_tool_url).query)

# Get submission result URL
url = urlparse(s.preview_url)
print(url.scheme + "://" + url.netloc + url.path)

## Download all submission responses

Iterates over all submissions and creates a table with the following columns:

user_id | assignment_id | submission_id | status | item_id | item_order | item_choice_choice | correct

This code retrieves the item responses for all the submissions listed in `<quiz_name>_submissions.csv`. If you only want to retrieve a subset of responses (maybe you had a few failures and only want to re-retrieve the ones that failed), just delete the rows for which you don't want to retrieve responses.

In [ ]:
# Set verbose = True to print information about every submission as it is retrieved.
# Information will be written to the log file regardless of this setting
verbose = False

auth_url =         'https://cornell.quiz-lti-iad-prod.instructure.com/api/participant_sessions/{:s}/grade'
quiz_session_url = 'https://cornell.quiz-api-iad-prod.instructure.com/api/quiz_sessions/{:s}'
results_url =      'https://cornell.quiz-api-iad-prod.instructure.com/api/quiz_sessions/{:s}/results/{:s}/session_item_results'

# Get bearer token (get from Chrome Developer Mode - good for 60 minutes)
with open('BEARER.txt', 'r') as f:
    bearer_token = f.readline()
    
with open('{:s}_responses.csv'.format(quiz_name), 'w', newline='') as out_file,\
     open('{:s}_log.txt'.format(quiz_name), 'a+') as log,\
     open('{:s}_submissions.csv'.format(quiz_name), 'r') as subs_file:

    log.write('Reading quiz responses...\n')
    
    n_subs = sum(1 for row in subs_file) - 1
    n_err = 0
    
    subs_file.seek(0)
    submissions = csv.DictReader(subs_file)
    
    fout = csv.DictWriter(out_file,
                          fieldnames=['user_id', 'assignment_id', 'submission_id', 'status',
                                      'item_id', 'item_order', 'item_choice_id', 'correct'])
    fout.writeheader()
    
    for s_i, s in enumerate(submissions):
        
        msg1 = 'User ID {:6d}, Submission ID {:7d} ...'.format(int(s['user_id']), int(s['submission_id']))
        log.write(msg1)
        if verbose:
            print(msg1, end='')
        else:
            print('{:3.0%} ... {:d} errors'.format((s_i+1) / n_subs, n_err), end='\r', flush=True)
        
        # Get an authorization token for this grading session
        r_auth = requests.get(url=auth_url.format(s['participant_session_id']),
                              headers={'authorization': 'Bearer {:s}'.format(bearer_token)})
        
        if r_auth.status_code == requests.codes.ok and 'token' in r_auth.json():
            auth_token = r_auth.json()['token']
        else:
            n_err += 1
            log.write('\n *** FAILED TO GET AUTH TOKEN\n')
            log.write(' {:d}\n'.format(r_auth.status_code))
            log.write(' ' + r_auth.text + '\n\n')
            
            if verbose:
                print('\n *** FAILED TO GET AUTH TOKEN')

            fout.writerow({'user_id': s['user_id'], 'assignment_id': s['assignment_id'],
                           'submission_id': s['submission_id'], 'status': 'FAILED'})
            continue
        
        # Get quiz submission details
        r_sub = requests.get(url=quiz_session_url.format(s['quiz_session_id']),
                             headers={'authorization': auth_token})
        
        if r_sub.status_code == requests.codes.ok and 'authoritative_result' in r_sub.json():
            result_id = r_sub.json()['authoritative_result']['id']
        else:
            n_err += 1
            log.write('\n *** FAILED TO GET SUBMISSION INFORMATION\n')
            log.write(' {:d}\n'.format(r_sub.status_code))
            log.write(' ' + r_sub.text + '\n\n')
            
            if verbose:
                print('\n *** FAILED TO GET SUBMISSION INFORMATION')
            
            fout.writerow({'user_id': s['user_id'], 'assignment_id': s['assignment_id'],
                           'submission_id': s['submission_id'], 'status': 'FAILED'})
            continue
        
        # Get student responses
        r_res = requests.get(url=results_url.format(s['quiz_session_id'], result_id),
                             headers={'authorization': auth_token})
        
        if r_res.status_code == requests.codes.ok:
            
            # Loop over quiz items
            for i, item in enumerate(r_res.json()):

                # Loop over item choices to determine which was chosen by student
                selected_choice = ''
                for item_choice in item['scored_data']['value']:
                    if item['scored_data']['value'][item_choice]['user_responded']:
                        item_choice_selected = item_choice

                fout.writerow({'user_id': s['user_id'],
                               'assignment_id': s['assignment_id'],
                               'submission_id': s['submission_id'],
                               'status': 'OK',
                               'item_id': item['item_id'],
                               'item_order': i+1,
                               'item_choice_id': item_choice_selected,
                               'correct': item['scored_data']['correct']})
            log.write(' OK\n')
            
            if verbose:
                print(' OK')

        else:
            n_err += 1
            log.write('\n *** FAILED TO GET RESULT INFORMATION\n')
            log.write(' {:d}\n'.format(r_res.status_code))
            log.write(' ' + r_res.text + '\n\n')
            
            if verbose:
                print('\n *** FAILED TO GET RESULT INFORMATION')
            
            fout.writerow({'user_id': s['user_id'], 'assignment_id': s['assignment_id'],
                           'submission_id': s['submission_id'], 'status': 'FAILED'})
            continue
            
print('Completed with {:d} errors.'.format(n_err))

## Get answer options

Get all the possible answer options for each question. Creates a file `quiz_<assignment_id>_item_options.csv` with the following columns:

assignment_id | item_id | item_name | item_text | item_choice_id | correct | item_choice_text

In [ ]:
# Get first valid submission
for s in quiz.get_submissions():
    if not s.missing:
        break

quiz_session = parse_qs(urlparse(s.external_tool_url).query)

# Get bearer token (get from Chrome Developer Mode - good for 60 minutes)
with open('BEARER.txt', 'r') as f:
    bearer_token = f.readline()

# Request to determine the quiz_session_id (not needed anymore)
url = 'https://cornell.quiz-lti-iad-prod.instructure.com/api/participant_sessions/{:s}/grade'\
    .format(quiz_session['participant_session_id'][0])

r_auth = requests.get(url, headers={'authorization': 'Bearer {:s}'.format(bearer_token)})

if r_auth.status_code == requests.codes.ok and 'token' in r_auth.json():
    auth_token = r_auth.json()['token']
else:
    print('Failed to get auth token')
    print(r_auth)
    r_auth.raise_for_status()


session_items_url = 'https://cornell.quiz-api-iad-prod.instructure.com/api/quiz_sessions/{:s}/session_items'

r_session = requests.get(url=session_items_url.format(quiz_session['quiz_session_id'][0]),
                         headers={'authorization': auth_token})

with open('{:s}_item_options.csv'.format(quiz_name), 'w', newline='') as out_file:

    fout = csv.DictWriter(out_file,
                          fieldnames=['assignment_id', 'item_id', 'item_name', 'item_text',
                                      'item_choice_id', 'correct', 'item_choice_text'])
    fout.writeheader()

    for item in r_session.json():
        for choice in item['item']['interaction_data']['choices']:
            fout.writerow({
                'assignment_id': assignment_id,
                'item_id': item['item']['id'],
                'item_name': item['item']['title'],
                'item_text': item['item']['item_body'],
                'item_choice_id': choice['id'],
                'item_choice_text': choice['item_body'],
                'correct': choice['id'] == item['scoring_data']['value']
            })
    
print('Done')